# Visualizations

<hr style="border:3px solid black"> </hr>

# Formatting Command

<hr style="border:1px solid black"> </hr>

## | fields 
   - allows you to include or exclude specific fields from search results
   - **useful**:
       - makes a search faster and more efficient 
       - limits the number of fields displayed
    
<br>

**EXAMPLE 1**:
<br>
| fields product_name price
<br>
^ field command, ^ desired fields to include

<br>

### Excluding field
use minus sign (-) between fields command and field names
<br>

**EXAMPLE 2**:
<br>
| fields - product_name price
<br>
**returns**: all fields EXCEPT product_name and price

<br>

**MUST** have a space after the minus sign or that command will not work
<br>

**EXAMPLE 3**:
<br>
| fields -product_name price
<br>
**returns**: ONLY returns price field

<br>

**| fields command defaults to inclusion**

---

## | table

- similar to fields command
- specified fields are included or excluded
- table command is a **transforming** command
- retains the data in a tabulated format

<br>

- Column Headers = field names
- Rows = values
- Each row = one event

<br>

- **Rearrange columns**: return to search and change order of fields inputted

**EXAMPLE**:
- | table         JSESSIONID   product_name price
- ^table command, ^session id, ^desired fields
- **returns**: the products that the specific user purchased and for what price

---

## | dedup

removes duplicate events from a table

**EXAMPLE**:
- | fields JSESSIOJNID price product_name
- | table JSESSIOJNID price product_name
- | dedup JSESSIOJNID price 

<br>

**Returns**:
   - filtered to include only 3 columns (fields)
   - turned the results into a table
   - removed all duplicate user IDs

---

## | addtotals

by default: computes the sum of all numberic fields for each row and creates a new "total" column

**EXAMPLE**:
- | chart sum(price) over product_name by VendorCountry
- | addtotals col=true label="Total Sales" labelfield="product_name" Fieldname="Total By Product"

<br>

**Returns**:
   - new column with total price by product_name ("Total By Product")
   - new row with total sales by country ("Total Sales")

---

## | fieldformat

- format appearance of values without making a change to underlying raw data
- creates new values, underlying data in index does **NOT** change

**EXAMPLE**:
- | fieldformat Total = "$" + tostring(Total, "commas")

**Returns**:
   - overwrite existing values in Total field 
   - turn into currency, using $ and commas

<hr style="border:2px solid black"> </hr>

# Visualizing Data

- any search that returns statistical values can be viewed as a chart
- most visualization require results structured as tables with AT LEAST two columns
- y axis should always be numeric

**EXAMPLE**:
| stats count(linecount) as "Units Sold" by product_name

<br>

**Returns**:
- total number of units sold per product_name

### Steps to visualization:
- Step 1: complete a search
- Step 2: click "visualization" tab (next to statistics tab)
- Step 3: click visualization menu ("line chart") and select type of visual desired (column graph, pie chart, etc)

**Single Series**:
- a table with only two columns
    - first column- X axis
    - second column - y axis

<hr style="border:2px solid black"> </hr>

# Tranforming Commands
needed to transform search results into visualizations

- | top
- | rare
- | stats
- | chart
- | timechart
- | trendline

---

## | top
- gives the most command values of given fields in a results set
- automatically returns count and percent columns
- limits to top 10 by default
    - can use "limit" clause to change the default return
    - use limit=0 to include ALL (no limit)
    
<br>

**EXAMPLE 1**:
- index=sales sourcetype=vendor_sales
- | top Vendor
<br>

**Returns**:
- which top 10 vendor had the highest sales for that period

<br>

**EXAMPLE 2**:
- index=sales sourcetype=vendor_sales
- | top Vendor limit=20
<br>

**Returns**:
- which top 20 vendor had the highest sales for that period

### top command clauses
- limit = int
    - how many results returned
- countfield = string
    - change name of count column to a string
- percentfield = string
    - change name of percent column to a string
- showcount = True/False
    - show the count column (yes or no)
- showperc = True/False
    - show the percent column (yes or no)
- showother = True/False
- otherstr = string
    - create row for all "other" that do not fall into limit

**EXAMPLE 3**:
- index=sales sourcetype=vendor_sales
- | top Vendor limit=5 showperc=false countfield="Number of Sales" useother=true
<br>

**Returns**:
   - top 5 vendors
   - remove percent column
   - change count column name to "number of sales"
   - create new row for all other vendors not listed in top 5

## "by" clause
top results split by another field

**EXAMPLE 4**:
- index=sales sourcetype=vendor_sales
- | top product_name by Vendor limit=3 countfield="Number of Sales" showperc=false

<br>

**Returns**:
   - top 3 product sold by each vendor in last 7 days
   - rename count column to "number of sales"
   - remove percent column

---

## | rare
shows least common values of a field set

**EXAMPLE**:
- index=sales sourcetype=vendor_sales
- | rare Vendor limit=3 countfield="Number of Sales" useother=true

<br>

**Returns**:
   - bottom 3 vendors 
   - rename count column to "number of sales"
   - create row for all other vendors not in bottom 3 limit

---

## | stats
- produces statistics from our search results
- do not need data to be time based

### Common Stats Functions
- count
- distinct count
- sum
- average
- min
- max
- list
- values

#### | stats count(field)
add field as an argument

---

## | chart
- takes two clause statements
    - over: tells Splunk which fields you want on the x-axis
    - by: create columns for each field
        - only one value can be specified
- any stats function can be applied to the chart command
- **remove nulls**: usenull=f
- limited to 10 arguments by default
- useother default to true (groups all "others" together)
- x-axis is NOT time

**EXAMPLE**:
<br>
index=web sourcetype=access_combined status>299
<br>
| chart count over status

---

## | timechart
- performs stats aggreggations against time
- time is always x-axis
- any stats function can be applied
- only one value can be be specified after "by" clause
- default to cluster in 24 hours groups
    - span=12hrs (or desired timeframe) <-- to change default

**EXAMPLE 1**:
<br>
index=sales sourcetype=vendor_sales
<br>
| timechart count by product_name

<br>

**Returns**:
- vendor sales over time
- timelines for each product_name

<br>

**EXAMPLE 2**:
<br>
index=sales sourcetype=vendor_sales
<br>
| timechart span=12h sum(price) by product_name limit=0

<br>

**Returns**:
- vendor sales by product_name over 12 hour time indevals

---

## | trendline
- computes moving averages of field values
- trendline command requires 3 arguments   

### Trendtype
- simple moving average (sma)
- exponential moving average (ema)
- weighted moving average (wma)

**EXAMPLE 1**:
<br>
index=web sourcetype=access_combined action=purchase status=200
<br>
| timechart sum(price) as sales
| trendline wma2(sales) as trend

<br>

**Returns**:
- sum of all sales with alias "sales"
- weighted moving average (2 days) with alias "trend"

---